In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

In [4]:
# import time series csv and SVI csv
case_series_df = pd.read_csv("resources/altered_data/time_series_cases_state.csv", encoding="ISO-8859-1")
SVI_df = pd.read_csv("resources/altered_data/SVI2018_US_COUNTY.csv", encoding="ISO-8859-1")

In [5]:
case_series_df

,UID,FIPS,COUNTY,STATE,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21
0,84001001,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,6517,6525,6533,6540,6543,6562,6570,6577,6580,6589
1,84001003,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,20361,20354,20395,20417,20423,20453,20473,20487,20492,20505
2,84001005,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,2213,2213,2216,2218,2221,2224,2226,2226,2227,2227
3,84001007,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,2529,2530,2535,2534,2535,2535,2536,2536,2537,2542
4,84001009,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,6387,6388,6402,6408,6415,6420,6424,6426,6443,6444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3108,84056037,56037.0,Sweetwater,Wyoming,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,0,...,3947,3951,3953,3961,3990,3990,3990,4002,4006,4022
3109,84056039,56039.0,Teton,Wyoming,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,3530,3532,3547,3549,3562,3562,3562,3593,3593,3609
3110,84056041,56041.0,Uinta,Wyoming,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,2109,2111,2113,2115,2119,2119,2119,2123,2125,2128
3111,84056043,56043.0,Washakie,Wyoming,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,889,889,890,890,890,890,890,891,891,891


In [29]:
SVI_df

,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,20044,12398,-999,...,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,23315,21115,8422,...,0,0,0,0,0,3875,508,7.1,0.9,37301
2,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,24222,20600,8220,...,0,0,0,0,0,6303,732,11.0,1.3,40036
3,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,10026,6708,4640,...,0,0,0,0,0,2005,340,10.2,1.7,17280
4,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,53682,45033,20819,...,0,0,0,0,0,10686,796,9.4,0.7,117894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,TEXAS,TX,Duval,48131,"Duval County, Texas",1793.476183,11355,5592,3511,2751,...,1,1,0,2,10,2656,406,24.5,3.7,9592
3138,TEXAS,TX,Zapata,48505,"Zapata County, Texas",998.411980,14369,6388,4405,5609,...,1,0,0,2,10,4206,484,29.3,3.4,10924
3139,TEXAS,TX,Zavala,48507,"Zavala County, Texas",1297.406535,12131,4344,3509,4150,...,1,1,0,2,10,2194,354,18.4,3.0,10672
3140,TEXAS,TX,Dimmit,48127,"Dimmit County, Texas",1328.884075,10663,4408,3309,3148,...,1,1,0,2,11,2345,539,22.2,5.1,8229


In [7]:
#merge on FIPS and then save to a csv file
SVI_case_series_df = pd.merge(case_series_df,SVI_df, on="FIPS", how="outer")
SVI_case_series_df

,UID,FIPS,COUNTY_x,STATE_x,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,84001001.0,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3875.0,508.0,7.1,0.9,37301.0
1,84001003.0,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,20864.0,1646.0,10.2,0.8,195677.0
2,84001005.0,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,8.0,2558.0,363.0,11.2,1.6,25052.0
3,84001007.0,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,1619.0,396.0,7.9,1.9,17696.0
4,84001009.0,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6303.0,732.0,11.0,1.3,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,51640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,5.0,973.0,329.0,15.3,5.2,10411.0
3148,NaN,51710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,3.0,5.0,28169.0,1505.0,12.9,0.7,260366.0
3149,NaN,51720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,5.0,397.0,140.0,10.1,3.6,6160.0
3150,NaN,51730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,2.0,6.0,4069.0,588.0,13.2,1.9,32249.0


In [11]:
SVI_case_series_df.isna().sum()

UID           39
FIPS           3
COUNTY_x      39
STATE_x       39
Lat           39
              ..
E_UNINSUR     10
M_UNINSUR     10
EP_UNINSUR    10
MP_UNINSUR    10
E_DAYPOP      10
Length: 491, dtype: int64

In [8]:
SVI_case_series_df.to_csv("SVI_case_series_merge.csv")

In [31]:
#load updated SVI_case_series_df
updated_SVI_case_series = pd.read_csv("resources/merged_data/SVI_case_series_merge_updated.csv")
updated_SVI_case_series

,FIPS,COUNTY,ST_ABBR,STATE,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,1001.0,Autauga,AL,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,3875.0,508.0,7.1,0.9,37301.0
1,1003.0,Baldwin,AL,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,0.0,0.0,0.0,1.0,1.0,20864.0,1646.0,10.2,0.8,195677.0
2,1005.0,Barbour,AL,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,0.0,0.0,1.0,2.0,8.0,2558.0,363.0,11.2,1.6,25052.0
3,1007.0,Bibb,AL,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,0.0,0.0,1.0,2.0,2.0,1619.0,396.0,7.9,1.9,17696.0
4,1009.0,Blount,AL,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,6303.0,732.0,11.0,1.3,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,56037.0,Sweetwater,WY,Wyoming,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,1.0,5240.0,794.0,12.0,1.8,45734.0
3108,56039.0,Teton,WY,Wyoming,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,1.0,0.0,0.0,1.0,2.0,2305.0,603.0,10.0,2.6,29426.0
3109,56041.0,Uinta,WY,Wyoming,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,1.0,2499.0,452.0,12.2,2.2,19734.0
3110,56043.0,Washakie,WY,Wyoming,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,1223.0,245.0,15.4,3.1,7859.0


In [32]:
updated_SVI_case_series.isna().sum()

FIPS          3
COUNTY        0
ST_ABBR       9
STATE         0
Lat           0
             ..
E_UNINSUR     9
M_UNINSUR     9
EP_UNINSUR    9
MP_UNINSUR    9
E_DAYPOP      9
Length: 487, dtype: int64

In the future need to go back and find those missing values. For now create new dataframe and put "0" for those null values

In [33]:
SVI_cases_df = updated_SVI_case_series.fillna("0")
SVI_cases_df.isna().sum()

FIPS          0
COUNTY        0
ST_ABBR       0
STATE         0
Lat           0
             ..
E_UNINSUR     0
M_UNINSUR     0
EP_UNINSUR    0
MP_UNINSUR    0
E_DAYPOP      0
Length: 487, dtype: int64

In [34]:
SVI_cases_df.dtypes

FIPS           object
COUNTY         object
ST_ABBR        object
STATE          object
Lat           float64
               ...   
E_UNINSUR      object
M_UNINSUR      object
EP_UNINSUR     object
MP_UNINSUR     object
E_DAYPOP       object
Length: 487, dtype: object


["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT	F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"] 

Rows that need to be converted to dtypes


In [36]:
#change dtypes
SVI_cases_df[["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"]] = SVI_cases_df[["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"]].astype('int')

START REGRESSION WITH CURRENT DATA (more data will be included in the future)

In [22]:
# view correlation tables 
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = SVI_cases_df.corr()
print (corrMatrix)

              Lat     Long_    3/1/20    3/2/20    3/3/20    3/4/20    3/5/20  \
Lat      1.000000 -0.345351  0.024979  0.031671  0.030620  0.030180  0.034020   
Long_   -0.345351  1.000000 -0.090448 -0.086472 -0.077636 -0.068716 -0.059899   
3/1/20   0.024979 -0.090448  1.000000  0.967445  0.944113  0.888950  0.846817   
3/2/20   0.031671 -0.086472  0.967445  1.000000  0.981647  0.925939  0.894009   
3/3/20   0.030620 -0.077636  0.944113  0.981647  1.000000  0.950068  0.920339   
...           ...       ...       ...       ...       ...       ...       ...   
3/27/21 -0.069413  0.006491  0.267093  0.226722  0.186161  0.264312  0.253710   
3/28/21 -0.069373  0.006696  0.266999  0.226654  0.186108  0.264203  0.253645   
3/29/21 -0.069234  0.007032  0.267022  0.226739  0.186219  0.264248  0.253741   
3/30/21 -0.069107  0.007290  0.267002  0.226752  0.186243  0.264195  0.253729   
3/31/21 -0.068974  0.007573  0.267064  0.226883  0.186372  0.264250  0.253817   

           3/6/20    3/7/20